In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import glob
import os
import subprocess
import sys
import scipy.io as spio
from scipy.stats import wilcoxon
from scipy.stats import anderson_ksamp
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt
#from mvpa2.tutorial_suite import *
from scipy.stats import wilcoxon, ttest_ind

In [ ]:
def initialize():
    TR = 2.43
    window_size = 12
    working_dir = "../depict"
    nifti_root = glob.glob('../data')[0]
    sbjs = sorted(glob.glob('%(nifti_root)s/[s]*' % {"nifti_root": nifti_root}))# sorted(os.listdir(nifti_path)[:])
    sbjs = [s.replace(nifti_root, '') for s in sbjs]
    sbjs = [s.replace('/', '') for s in sbjs]
    nsbj = len(sbjs)    
    sbjid = np.arange(nsbj)

    return TR, window_size, working_dir, nifti_root, sbjs, sbjid

In [ ]:
def getpaths(isbj):
    TR, window_size, working_dir, nifti_root, sbjs, sbjid = initialize()
    sbj_nifti_path = '%(nifti_root)s/%(sbj)s' % \
        {"nifti_root": nifti_root, "sbj": sbjs[isbj]}
    sbj_path = '%(working_dir)s/%(sbj)s' % \
        {"working_dir": working_dir, "sbj": sbjs[isbj]}
      
    run_path = sorted(glob.glob('%(sbj_path)s/ep*' % \
        {"sbj_path": sbj_nifti_path}))
    nruns = len(run_path)
    mask_path = glob.glob('%(sbj_path)s/t1*/brain.nii' % \
        {"sbj_path": sbj_nifti_path})
    return sbj_nifti_path, sbj_path, run_path, nruns, mask_path

In [ ]:
def write_conditions():
    TR, window_size, working_dir, nifti_root, sbjs, sbjid = initialize()
    dscv = np.zeros((35, 6)).astype(int)
    latch = np.zeros(21).astype(int)
    dscv[:,0] = np.array([6,3,3,3,5,3,2,8,1,7,1,1,1,1,1,1,8,3,1,1,2,3,2,3,3,3,5,7,3,5,9,9,18,10,41])-1 # understand 11th number
    dscv[:,1] = np.array([10,7,9,10,7,3,6,15,3,7,14,12,18,2,14,5,14,11,10,4,2,7,11,8,9,41,1,41,41,18,41,41,41,41,41])-1 #understand 7th number
    dscv[:,2] = np.array([15,7,8,9,14,17,12,15,19,6,14,18,1,6,11,17,11,4,19,4,2,16,1,41,41,41,41,41,41,41,41,41,41,41,41])-1 # understand 3th number
    dscv[:,3] = np.array([24,21,22,21,23,23,21,22,22,21,21,24,21,21,21,23,23,21,21,21,21,24,1,26,26,22,25,21,24,23,27,31,23,21,23])-1 # understand 12th number
    dscv[:,4] = np.array([28,21,23,24,25,33,21,24,25,21,21,22,21,21,23,23,23,23,21,21,21,24,29,26,41,35,25,22,41,28,41,41,41,41,41])-1 # understand 7th number
    dscv[:,5] = np.array([29,21,24,25,26,33,23,23,36,21,21,27,27,21,36,23,26,24,37,21,22,21,40,38,35,41,41,41,41,40,41,41,41,41,41])-1 # understand 3th number
    # number of the game in which subject underestands the sterategy
    latch = np.array([29,6,8,24,25,33,9,23,35,6,14,24,27,4,31,23,24,23,36,3,5])-1 
    
    # CONSTRUCTING THE HEADER 
    f = open('conditions_file_nim.txt', 'a')
    f.write('condition_name , subject_number, session_number, onsets, durations\n')
    f.close()

    
    #writing to a text file:
    with open('conditions_file_nim.txt', 'a') as f:

      
    
      for jsbj in range(len(sbjid)):

          print(jsbj)         
          isbj = sbjid[jsbj]
          sbj_nifti_path, sbj_path, run_path, nruns, mask_path = getpaths(isbj)
            
          if isbj == 16:
              f.write('rest2,' + str(isbj) + ',4, 1, 170' + '\n')
          else :
              # rest1 (before starting the experiment :run 1)
              f.write('rest1,' + str(isbj) + ',1, 1, 170' + '\n')
              # rest2 (at the end, after experiment :run 4)
              f.write('rest2,' + str(isbj) + ',4, 1, 170' + '\n')
            
            #loading the .mat data (which include onsets and other informations)
          matdata = spio.loadmat('../behaviour/sbj%(sbj)02d.mat' % {"sbj": isbj+1}) 


            
          #print(run_path[-2])
          #print(run_path[-1])
          scanonset = np.zeros(40).astype(float)

          scanoffset = np.zeros(40).astype(float)

          sbjmove = np.zeros(6).astype(int)


            
          for igame in range(40):
              if igame < 20:
                  #ipath = run_path[-2] #first task
                  if isbj == 12 or isbj == 21:
                      scanonset[igame] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,0] -  matdata['ExpInfo']['triggerstart']
                      
                      sbjmove[0] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart']
                      sbjmove[1] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-2] -  matdata['ExpInfo']['triggerstart']
                      sbjmove[2] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-3] -  matdata['ExpInfo']['triggerstart']

                      pcmove = matdata['ExpInfo']['game'][0,0]['offset_pc'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart']
                      scanoffset[igame] = np.maximum(sbjmove[0], pcmove)
                  else:
                      scanonset[igame] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,0] -  matdata['ExpInfo']['triggerstart1']

                      sbjmove[0] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart1']
                      sbjmove[1] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-2] -  matdata['ExpInfo']['triggerstart1']
                      sbjmove[2] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-3] -  matdata['ExpInfo']['triggerstart1']

                      pcmove = matdata['ExpInfo']['game'][0,0]['offset_pc'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart1']

                      scanoffset[igame] = np.maximum(sbjmove[0], pcmove)
                        
              else: #second task (20-39 games) it's the 3rd run
                  #ipath = run_path[-1] #second task

                  scanonset[igame] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,0] -  matdata['ExpInfo']['triggerstart2']
                  #trial = len(matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,:]) - 1
                  #sbjmove = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart2']
                  sbjmove[3] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart2']
                  sbjmove[4] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-2] -  matdata['ExpInfo']['triggerstart2']
                  sbjmove[5] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-3] -  matdata['ExpInfo']['triggerstart2']

                  pcmove = matdata['ExpInfo']['game'][0,0]['offset_pc'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart2']
                  
                  scanoffset[igame] = np.maximum(sbjmove[3], pcmove)



             #______________________________________________________________________
             # LATCH condtion (subjects who have latch)

          if isbj<= len(latch):

              # SEARCH CONDITION
              session_num = 3
              if latch[isbj-1] < 20: #if latching happened in the task 1 
                session_num = 2

                
                DURATIONS = []
                ONSETS = []
                for num in range(latch[isbj-1]):
                    DURATIONS = DURATIONS + [scanoffset[num]- scanonset[num]]

                    ONSETS = ONSETS + [scanonset[num]]

                f.write('search1,'+ str(isbj) +str(session_num) + str(ONSETS)+ str(DURATIONS) + '\n')

              else:
                  ONSETS = []
                  DURATIONS = []

                  for i in range(20):
                      ONSETS = ONSETS + [scanonset[i]]
                      DURATIONS = DURATIONS + [scanoffset[i]- scanonset[i]] 

                  f.write('search1,'+ str(isbj) +str(2) + str(ONSETS)+ str(DURATIONS) + '\n')

                  ONSETS = []
                  DURATIONS = []

                  for num in range(20 ,latch[isbj-1] ):
                      DURATIONS = DURATIONS + [scanoffset[num]- scanonset[num]]

                      ONSETS = ONSETS + [scanonset[num]]    

                  f.write('search2,'+ str(isbj) +str(3) + str(ONSETS)+ str(DURATIONS) + '\n')


              #LATCH condition 
              duration = scanoffset[latch[isbj-1]] - scanonset[latch[isbj-1]] 


              f.write('latch,'+ str(isbj) +str(session_num) + str(scanonset[latch[isbj-1]])+ str(duration) + '\n')


              #Before latch :

              duration = scanoffset[latch[isbj-1]-1] - scanonset[latch[isbj-1]-1] 
              f.write('before_latch,'+ str(isbj) +str(session_num) + str(scanonset[latch[isbj-1]-1])+ str(duration) + '\n')

                #FOLLOWING condition

              session_num = 3
              if latch[isbj-1] < 20: #if latching happened in the task 1 
                session_num = 2

                
                DURATIONS = []
                ONSETS = []
                for num in range(latch[isbj-1]+1,20):
                  DURATIONS = DURATIONS + [scanoffset[num]- scanonset[num]]

                  ONSETS = ONSETS + [scanonset[num]]

                f.write('follow 1,'+ str(isbj) +str(session_num) + str(ONSETS)+ str(DURATIONS) + '\n')
                for num in range(20 ,40):
                  DURATIONS = DURATIONS + [scanoffset[num]- scanonset[num]]

                  ONSETS = ONSETS + [scanonset[num]]    

                f.write('follow 2,'+ str(isbj) +str(3) + str(ONSETS)+ str(DURATIONS) + '\n')

                #APPLY condition
                f.write('apply,'+ str(isbj) +str(3) + str(scanonset[20])+ str(scanoffset[20]- scanonset[20]) + '\n')


              else:
                ONSETS = []
                DURATIONS = []

                for num in range(latch[isbj-1]+1 , 40 ):
                  DURATIONS = DURATIONS + [scanoffset[num]- scanonset[num]]

                  ONSETS = ONSETS + [scanonset[num]]    

                f.write('follow 2,'+ str(isbj) +str(3) + str(ONSETS)+ str(DURATIONS) + '\n')

                    #______________________________________________________________________

                  
            #__________________________________________________________________________
            #subjects without latch
          else:
              ONSETS = []
              DURATIONS = []

              for i in range(20):
                  ONSETS = ONSETS + [scanonset[i]]
                  DURATIONS = DURATIONS + [scanoffset[i]- scanonset[i]] 

              f.write('search1,'+ str(isbj) +str(2) + str(ONSETS)+ str(DURATIONS) + '\n')

              ONSETS = []
              DURATIONS = []
              for num in range(20 ,40 ):
                  DURATIONS = DURATIONS + [scanoffset[num]- scanonset[num]]

                  ONSETS = ONSETS + [scanonset[num]]    

              f.write('search2,'+ str(isbj) +str(3) + str(ONSETS)+ str(DURATIONS) + '\n')

                  



###################### DISCOVERY:
          for golden_num in range(3): #for first game (15 steps)

            if dscv[isbj,golden_num ] != 40:
              if isbj == 12 or isbj == 21:
                sbjmove_onset = matdata['ExpInfo']['game'][0,0]['onset'][0, dscv[isbj,golden_num ]][0,(-golden_num -1)] -  matdata['ExpInfo']['triggerstart']
                sbjmove_offset = matdata['ExpInfo']['game'][0,0]['offset'][0, dscv[isbj,golden_num]][0,(-golden_num -1)] -  matdata['ExpInfo']['triggerstart']

              else:
                sbjmove_onset = matdata['ExpInfo']['game'][0,0]['onset'][0, dscv[isbj,golden_num ]][0,(-golden_num -1)] -  matdata['ExpInfo']['triggerstart1']
                sbjmove_offset = matdata['ExpInfo']['game'][0,0]['offset'][0, dscv[isbj,golden_num]][0,(-golden_num -1)] -  matdata['ExpInfo']['triggerstart1']

              DURATIONS = sbjmove_offset - sbjmove_onset
              f.write('discovery{},'.format(golden_num)+ str(isbj) +str(2) + str(sbjmove_onset)+ str(DURATIONS) + '\n')

          for golden_num in range(3,6): #for second game (17 steps)

            if dscv[isbj,golden_num ] != 40:
              if isbj == 12 or isbj == 21:
                sbjmove_onset = matdata['ExpInfo']['game'][0,0]['onset'][0, dscv[isbj,golden_num ]][0,(-golden_num +2)] -  matdata['ExpInfo']['triggerstart']
                sbjmove_offset = matdata['ExpInfo']['game'][0,0]['offset'][0, dscv[isbj,golden_num]][0,(-golden_num +2)] -  matdata['ExpInfo']['triggerstart']
              else:
                sbjmove_onset = matdata['ExpInfo']['game'][0,0]['onset'][0, dscv[isbj,golden_num ]][0,(-golden_num +2)] -  matdata['ExpInfo']['triggerstart2']
                sbjmove_offset = matdata['ExpInfo']['game'][0,0]['offset'][0, dscv[isbj,golden_num]][0,(-golden_num +2)] -  matdata['ExpInfo']['triggerstart2']

              DURATIONS = sbjmove_offset - sbjmove_onset
              f.write('discovery{},'.format(golden_num)+ str(isbj) +str(3) + str(sbjmove_onset)+ str(DURATIONS) + '\n')


In [ ]:
# 

In [ ]:
write_conditions()


conditions: 1/ rest1 : subject number,1=run number, 1=onset, duration=170

2/ rest2 : subject number,4=run number, 1=onset, duration=170

3/ search 1 0r 2: before understanding (1 stands for game 1, 2 stands for game 2)

4/ latch : phase of understanding

5/ before latch: exactly before the latch time

6/ follow 1 or 2: after understanding the latching till the end of correspond task, depdens on the game 1 or 2

7/ apply : latche <= 20 equivalently the strategy is understood before game 1

 ... 

13/ Discoveries (consist of 6 states): for each 11, 7, 3, 12, 7, 3 golden numbers

# SBJMOVE:

in order to deteremine the duration time which correspond to Discoveries conditions, we have to use " sbjmove ".


# **Third version**
This is the third version of our conditions code which many of the errors regarding the subject numbers and the writing text style have been solved. it still has one error and we need to reduce the "[ ]" and ',' s in the arrays for onsets and durations columns.

In [ ]:
from this import d
import numpy as np
import pandas as pd
import seaborn as sns
import glob
import os
import subprocess
import sys
import scipy.io as spio
from scipy.stats import wilcoxon
from scipy.stats import anderson_ksamp
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt
#from mvpa2.tutorial_suite import *
from scipy.stats import wilcoxon, ttest_ind

def initialize():
    TR = 2.43
    window_size = 12
    working_dir = "../depict"
    nifti_root = glob.glob('../data')[0]
    sbjs = sorted(glob.glob('%(nifti_root)s/[s]*' % {"nifti_root": nifti_root}))# sorted(os.listdir(nifti_path)[:])
    sbjs = [s.replace(nifti_root, '') for s in sbjs]
    sbjs = [s.replace('/', '') for s in sbjs]
    nsbj = len(sbjs)    
    sbjid = np.arange(nsbj)

    return TR, window_size, working_dir, nifti_root, sbjs, sbjid


def getpaths(isbj):
    TR, window_size, working_dir, nifti_root, sbjs, sbjid = initialize()
    sbj_nifti_path = '%(nifti_root)s/%(sbj)s' % \
        {"nifti_root": nifti_root, "sbj": sbjs[isbj]}
    sbj_path = '%(working_dir)s/%(sbj)s' % \
        {"working_dir": working_dir, "sbj": sbjs[isbj]}
      
    run_path = sorted(glob.glob('%(sbj_path)s/ep*' % \
        {"sbj_path": sbj_nifti_path}))
    nruns = len(run_path)
    mask_path = glob.glob('%(sbj_path)s/t1*/brain.nii' % \
        {"sbj_path": sbj_nifti_path})
    return sbj_nifti_path, sbj_path, run_path, nruns, mask_path



def write_conditions():
    TR, window_size, working_dir, nifti_root, sbjs, sbjid = initialize()
    dscv = np.zeros((35, 6)).astype(int)
    latch = np.zeros(21).astype(int)
    dscv[:,0] = np.array([6,3,3,3,5,3,2,8,1,7,1,1,1,1,1,1,8,3,1,1,2,3,2,3,3,3,5,7,3,5,9,9,18,10,41])-1 # understand 11th number
    dscv[:,1] = np.array([10,7,9,10,7,3,6,15,3,7,14,12,18,2,14,5,14,11,10,4,2,7,11,8,9,41,1,41,41,18,41,41,41,41,41])-1 #understand 7th number
    dscv[:,2] = np.array([15,7,8,9,14,17,12,15,19,6,14,18,1,6,11,17,11,4,19,4,2,16,1,41,41,41,41,41,41,41,41,41,41,41,41])-1 # understand 3th number
    dscv[:,3] = np.array([24,21,22,21,23,23,21,22,22,21,21,24,21,21,21,23,23,21,21,21,21,24,1,26,26,22,25,21,24,23,27,31,23,21,23])-1 # understand 12th number
    dscv[:,4] = np.array([28,21,23,24,25,33,21,24,25,21,21,22,21,21,23,23,23,23,21,21,21,24,29,26,41,35,25,22,41,28,41,41,41,41,41])-1 # understand 7th number
    dscv[:,5] = np.array([29,21,24,25,26,33,23,23,36,21,21,27,27,21,36,23,26,24,37,21,22,21,40,38,35,41,41,41,41,40,41,41,41,41,41])-1 # understand 3th number
    # number of the game in which subject underestands the sterategy
    latch = np.array([29,6,8,24,25,33,9,23,35,6,14,24,27,4,31,23,24,23,36,3,5])-1 
    
    # CONSTRUCTING THE HEADER 
    f = open('conditions_file_nim.txt', 'a')
    f.write('condition_name,subject_number,session_number,onset,durations\n')
    f.close()


    #writing to a text file:
    with open('conditions_file_nim.txt', 'a') as f:

      
    
      for jsbj in range(len(sbjid)):

                   
          isbj = sbjid[jsbj]
          print(isbj)
          sbj_nifti_path, sbj_path, run_path, nruns, mask_path = getpaths(isbj)


          ## subject 16 has just one rest state there for: 
            
          if isbj+1 == 16:
              f.write('rest2,' + str(isbj+1) + ',4,1,170' + '\n')
          else :
              # rest1 (before starting the experiment :run 1)
              f.write('rest1,' + str(isbj+1) + ',1,1,170' + '\n')
              # rest2 (at the end, after experiment :run 4)
              f.write('rest2,' + str(isbj+1) + ',4,1,170' + '\n')
            
            #loading the .mat data (which include onsets and other informations)
          matdata = spio.loadmat('../behaviour/sbj%(sbj)02d.mat' % {"sbj": isbj+1}) 


            
          #print(run_path[-2])
          #print(run_path[-1])
          scanonset = np.zeros(40)

          scanoffset = np.zeros(40)

          sbjmove_offset = np.zeros(6)
          sbjmove_onset = np.zeros(6)


            
          for igame in range(40):
              if igame < 20:
                  #ipath = run_path[-2] #first task
                  # sbjmove_onnset[0] is the first step of the curent game which subject finds number 3 for the first 20 games 
                  # sbjmove_onnset[1] is the first step of the curent game which subject finds number 7 for the first 20 games
                  # sbjmove_onnset[2] is the first step of the curent game which subject finds number 11 for the firs 20 games    



                  if isbj == 12 or isbj == 21:
                      scanonset[igame] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,0] -  matdata['ExpInfo']['triggerstart']
                      
                      sbjmove_offset[0] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart']
                      sbjmove_offset[1] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-2] -  matdata['ExpInfo']['triggerstart']
                      sbjmove_offset[2] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-3] -  matdata['ExpInfo']['triggerstart']

                      sbjmove_onset[0] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart']
                      sbjmove_onset[1] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,-2] -  matdata['ExpInfo']['triggerstart']
                      sbjmove_onset[2] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,-3] -  matdata['ExpInfo']['triggerstart']


                      pcmove = matdata['ExpInfo']['game'][0,0]['offset_pc'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart']
                      scanoffset[igame] = np.maximum(sbjmove_offset[0], pcmove)
                  else:
                      scanonset[igame] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,0] -  matdata['ExpInfo']['triggerstart1']

                      sbjmove_offset[0] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart1']
                      sbjmove_offset[1] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-2] -  matdata['ExpInfo']['triggerstart1']
                      sbjmove_offset[2] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-3] -  matdata['ExpInfo']['triggerstart1']

                      sbjmove_onset[0] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart1']
                      sbjmove_onset[1] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,-2] -  matdata['ExpInfo']['triggerstart1']
                      sbjmove_onset[2] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,-3] -  matdata['ExpInfo']['triggerstart1']

                      pcmove = matdata['ExpInfo']['game'][0,0]['offset_pc'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart1']

                      scanoffset[igame] = np.maximum(sbjmove_offset[0], pcmove)
                        
              else: #second task (20-39 games) it's the 3rd run
                  #ipath = run_path[-1] #second task
                  # sbjmove_onnset[3] is the first step of the curent game which subject finds number 3  for the last 20 games
                  # sbjmove_onnset[4] is the first step of the curent game which subject finds number 7 for the last 20 games
                  # sbjmove_onnset[5] is the first step of the curent game which subject finds number 11 for the last 20 games (subject's turn to opt the golden number  )

                  scanonset[igame] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,0] -  matdata['ExpInfo']['triggerstart2']
                  #trial = len(matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,:]) - 1
                  #sbjmove = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart2']
                  sbjmove_offset[3] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart2']
                  sbjmove_offset[4] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-2] -  matdata['ExpInfo']['triggerstart2']
                  sbjmove_offset[5] = matdata['ExpInfo']['game'][0,0]['offset'][0, igame][0,-3] -  matdata['ExpInfo']['triggerstart2']

                  sbjmove_onset[3] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart2']
                  sbjmove_onset[4] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,-2] -  matdata['ExpInfo']['triggerstart2']
                  sbjmove_onset[5] = matdata['ExpInfo']['game'][0,0]['onset'][0, igame][0,-3] -  matdata['ExpInfo']['triggerstart2']

                  pcmove = matdata['ExpInfo']['game'][0,0]['offset_pc'][0, igame][0,-1] -  matdata['ExpInfo']['triggerstart2']
                  
                  scanoffset[igame] = np.maximum(sbjmove_offset[3], pcmove)

              
              ###################### DISCOVERY:
              DURATIONS = 0
              for golden_num in range(6):
                  if dscv[isbj,golden_num ] != 40:
                      if dscv[isbj,golden_num] == igame :
                          DURATIONS = sbjmove_offset[golden_num] - sbjmove_onset[golden_num]
                          session_num = 3
                          if igame < 20:  
                            session_num = 2
                          f.write('discovery{},'.format(golden_num)+ str(isbj+1) +',{},'.format(session_num) + str(sbjmove_onset[golden_num])+',' + str(DURATIONS) + '\n')
                          




          #______________________________________________________________________
          # LATCH condtion (subjects who have latch)

          if isbj<= len(latch)-1:

              # SEARCH CONDITION
              session_num = 3
              if latch[isbj] < 20: #if latching happened in the task 1 
                session_num = 2

                
                DURATIONS = np.zeros(latch[isbj]-1)
                ONSETS = np.zeros(latch[isbj]-1)
                for num in range(latch[isbj]-1):
                    DURATIONS[num] = scanoffset[num]- scanonset[num]

                    ONSETS[num] =scanonset[num]

                f.write('search1,'+ str(isbj+1) +',{},'.format(session_num) + str(ONSETS).replace('[ ', '').replace(']', '')+ ',' +str(DURATIONS).replace('[ ', '').replace(']', '') + '\n')

              else:
                  ONSETS = np.zeros(20)
                  DURATIONS = np.zeros(20)

                  for i in range(20):
                      ONSETS[i] = scanonset[i]
                      DURATIONS[i] = scanoffset[i]- scanonset[i]

                  f.write('search1,'+ str(isbj+1) + ',' +str(2) +',' +str(ONSETS).replace('[ ', '').replace(']', '')+ ',' +str(DURATIONS).replace('[ ', '').replace(']', '') + '\n')

                  ONSETS = np.zeros(latch[isbj]-20)
                  DURATIONS = np.zeros(latch[isbj]-20)

                  for num in range(20 ,latch[isbj] ):
                      DURATIONS[num-20] = scanoffset[num-20]- scanonset[num-20]

                      ONSETS[num-20] = scanonset[num-20]    

                  f.write('search2,'+ str(isbj+1) + ',' +str(3) +','+ str(ONSETS).replace('[ ', '').replace(']', '')+','+ str(DURATIONS).replace('[ ', '').replace(']', '') + '\n')


              #LATCH condition 
              duration = scanoffset[latch[isbj]] - scanonset[latch[isbj]] 


              f.write('latch,'+ str(isbj+1) + ',' + str(session_num) + ',' + str(scanonset[latch[isbj-1]])+ ',' + str(duration) + '\n')


              #Before latch :

              duration = scanoffset[latch[isbj]-1] - scanonset[latch[isbj]-1] 
              f.write('before_latch,'+ str(isbj+1) + ',' + str(session_num) + ',' + str(scanonset[latch[isbj]-1]) + ',' + str(duration) + '\n')

              #FOLLOWING condition

              session_num = 3
              if latch[isbj] < 20: #if latching happened in the task 1 
                session_num = 2

                
                DURATIONS = np.zeros(19 - latch[isbj])
                ONSETS = np.zeros(19 - latch[isbj])

                d = 19 - latch[isbj]
                for num in range(d):
                  DURATIONS[num] = scanoffset[num+ latch[isbj]+1]- scanonset[num +latch[isbj]+1]

                  ONSETS[num] = scanonset[num+latch[isbj]+1]

                f.write('follow1,'+ str(isbj+1) + ',' + str(session_num) + ',' + str(ONSETS).replace('[ ', '').replace(']', '')+ ',' + str(DURATIONS).replace('[ ', '').replace(']', '') + '\n')
                
                ONSETS = np.zeros(20)
                DURATIONS = np.zeros(20)
                for num in range(20 ,40):
                  DURATIONS[num -20] = scanoffset[num]- scanonset[num]

                  ONSETS[num-20] = scanonset[num]    

                f.write('follow2,'+ str(isbj+1) + ',' +str(3) + ',' + str(ONSETS).replace('[ ', '').replace(']', '') + ',' + str(DURATIONS).replace('[ ', '').replace(']', '') + '\n')

                #APPLY condition
                f.write('apply,'+ str(isbj+1) + ',' + str(3) + ',' + str(scanonset[20])+ ',' + str(scanoffset[20]- scanonset[20]) + '\n')


              else:
                ONSETS = np.zeros(39 - latch[isbj])
                DURATIONS = np.zeros(39 - latch[isbj])

                c = 39 - latch[isbj]

                for num in range(c):
                  DURATIONS[num] =scanoffset[num+ latch[isbj] +1 ]- scanonset[num+latch[isbj] + 1]

                  ONSETS[num] = scanonset[num + latch[isbj] +1]    

                f.write('follow2,'+ str(isbj+1) + ',' +str(3) + ',' + ONSETS.replace('[ ', '').replace(']', '')+ ',' + DURATIONS.replace('[ ', '').replace(']', '') + '\n')

                    #______________________________________________________________________

                  
            #__________________________________________________________________________
            #subjects without latch
          else:
              ONSETS = np.zeros(20)
              DURATIONS = np.zeros(20)

              for i in range(20):
                  ONSETS[i] = scanonset[i]
                  DURATIONS[i] = scanoffset[i]- scanonset[i]

              f.write('search1,'+ str(isbj+1) + ',' + str(2) + ',' + ONSETS.replace('[ ', '').replace(']', '')+ ',' + DURATIONS.replace('[ ', '').replace(']', '') + '\n')

              ONSETS = np.zeros(20)
              DURATIONS = np.zeros(20)
              for num in range(20 ,40 ):
                  DURATIONS[num-20] = scanoffset[num]- scanonset[num]

                  ONSETS[num-20] = scanonset[num]    

              f.write('search2,'+ str(isbj+1) + ',' +str(3) + ',' + ONSETS.replace('[ ', '').replace(']', '')+ ',' + DURATIONS.replace('[ ', '').replace(']', '') + '\n')

                  





write_conditions()


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


IndexError: ignored

In [ ]:

import numpy as np

p = np.array([[" asdf ", " asdf " ], [" afhffh hfhf ", " a "]])
print(type(p))
print(p)

p = np.char.strip(p)
print(p)

<class 'numpy.ndarray'>
[[' asdf ' ' asdf ']
 [' afhffh hfhf ' ' a ']]
[['asdf' 'asdf']
 ['afhffh hfhf' 'a']]


# Q uestions to ask 

In [ ]:
#aya hame latch ha vaghean be tartib subject ha hastan